In [1]:
import pandas as pd
import numpy as np

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
node_features =  pd.read_csv('../docs/node_features.csv')

In [4]:
source_features = pd.read_csv('../docs/source_features.csv.zip')

In [5]:
source_features.drop('Unnamed: 0',axis=1,inplace=True)

In [6]:
#source_features = source_features_start.join(node_features.set_index('node_id').drop(['is_parent_node','is_leaf_node','parents_count','children_links_counts',
                                                           #'parents_links_counts'], axis=1),on='node_id')

In [7]:
source_features.head()

,node_id,parents_count,non_leaf_descendants_count,leaf_descendans_count
0,1097,21,37.0,139.0
1,1122,109,251.0,761.0
2,1152,94,99.0,2678.0
3,1170,138,118.0,334.0
4,1174,187,198.0,344.0


In [8]:
source_features.nunique()

node_id                       20000
parents_count                   902
non_leaf_descendants_count      845
leaf_descendans_count          2891
dtype: int64

In [9]:
sink_features = pd.read_csv('../docs/sink_features.v2.csv.gz')

In [10]:
sink_features.drop('is_leaf_node',inplace=True,axis=1)

In [11]:
sink_features.count()

node_id                              4867136
parent_non_leaf_descendants_count    4867136
parent_leaf_descendans_count         4867136
parents_count                        4867136
dtype: int64

In [12]:
test_data = pd.read_csv('../docs/coef_comparision_TestDataSet.csv')
test_data.head()

,Source,Sink,AA,JA,PA,Id
0,2184483,1300190,0.000000,0.000000,435,1
1,3151356,1452193,0.407705,0.006260,102306,2
2,1579396,193159,0.000000,0.000000,418,3
3,1406432,2481036,1.238898,0.062500,2838,4
4,2389638,593017,0.802812,0.012072,62196,5


In [13]:
test_data.drop(['JA','PA'],axis=1,inplace=True)

# Feature matrix for test data

In [14]:
df1 = test_data.join(source_features.set_index('node_id'),on='Source')

In [15]:
df1.head()

,Source,Sink,AA,Id,parents_count,non_leaf_descendants_count,leaf_descendans_count
0,2184483,1300190,0.000000,1,102,65.0,18.0
1,3151356,1452193,0.407705,2,39,119.0,221.0
2,1579396,193159,0.000000,3,13,66.0,142.0
3,1406432,2481036,1.238898,4,16,48.0,36.0
4,2389638,593017,0.802812,5,30,69.0,198.0


In [16]:
df1.rename(columns= {'parents_count':'source_parents_count',
                              'non_leaf_descendants_count':'source_non_leaf_descendants_count',
                              'leaf_descendans_count':'source_leaf_descendans_count'},inplace=True)

In [17]:
df1.head()

,Source,Sink,AA,Id,source_parents_count,source_non_leaf_descendants_count,source_leaf_descendans_count
0,2184483,1300190,0.000000,1,102,65.0,18.0
1,3151356,1452193,0.407705,2,39,119.0,221.0
2,1579396,193159,0.000000,3,13,66.0,142.0
3,1406432,2481036,1.238898,4,16,48.0,36.0
4,2389638,593017,0.802812,5,30,69.0,198.0


In [18]:
df2 = test_data.join(sink_features.set_index('node_id'),on='Sink')

In [19]:
df2.rename(columns={'parents_count':'sink_parents_count',
                              'parent_non_leaf_descendants_count':'sink_parent_non_leaf_descendants_count',
                              'parent_leaf_descendans_count':'sink_parent_leaf_descendans_count'},inplace=True)

In [20]:
df2.head()

,Source,Sink,AA,Id,sink_parent_non_leaf_descendants_count,sink_parent_leaf_descendans_count,sink_parents_count
0,2184483,1300190,0.000000,1,2702,109609,3
1,3151356,1452193,0.407705,2,179071,8088742,289
2,1579396,193159,0.000000,3,3396,16755,2
3,1406432,2481036,1.238898,4,2283,4034,24
4,2389638,593017,0.802812,5,41352,544270,165


In [21]:
X_test = df1.join(df2.set_index('Id').drop(['Source','AA','Sink'],axis=1),on='Id')

In [22]:
X_test.head()

,Source,Sink,AA,Id,source_parents_count,source_non_leaf_descendants_count,source_leaf_descendans_count,sink_parent_non_leaf_descendants_count,sink_parent_leaf_descendans_count,sink_parents_count
0,2184483,1300190,0.000000,1,102,65.0,18.0,2702,109609,3
1,3151356,1452193,0.407705,2,39,119.0,221.0,179071,8088742,289
2,1579396,193159,0.000000,3,13,66.0,142.0,3396,16755,2
3,1406432,2481036,1.238898,4,16,48.0,36.0,2283,4034,24
4,2389638,593017,0.802812,5,30,69.0,198.0,41352,544270,165


In [23]:
X_test.columns

Index(['Source', 'Sink', 'AA', 'Id', 'source_parents_count',
       'source_non_leaf_descendants_count', 'source_leaf_descendans_count',
       'sink_parent_non_leaf_descendants_count',
       'sink_parent_leaf_descendans_count', 'sink_parents_count'],
      dtype='object')

# Caliculating rank of the matrix

In [24]:
from numpy.linalg import matrix_rank

In [25]:
X_test.head()

,Source,Sink,AA,Id,source_parents_count,source_non_leaf_descendants_count,source_leaf_descendans_count,sink_parent_non_leaf_descendants_count,sink_parent_leaf_descendans_count,sink_parents_count
0,2184483,1300190,0.000000,1,102,65.0,18.0,2702,109609,3
1,3151356,1452193,0.407705,2,39,119.0,221.0,179071,8088742,289
2,1579396,193159,0.000000,3,13,66.0,142.0,3396,16755,2
3,1406432,2481036,1.238898,4,16,48.0,36.0,2283,4034,24
4,2389638,593017,0.802812,5,30,69.0,198.0,41352,544270,165


In [26]:
matrix_rank(X_test.values[:, 3:])

7

In [27]:
0.

0.0

# Standardising the test data

In [28]:
from sklearn import preprocessing

In [29]:
#X_test_standardised = X_test.drop(['Id','Source','Sink','AA'],axis=1).apply(preprocessing.robust_scale)

In [30]:
#X_test_standardised = X_test.drop(['Id','Source','Sink'],axis=1).apply(lambda x: np.log10(x+1))

In [31]:
X_test_standardised = X_test.drop(['Id','Source','Sink'],axis=1).apply(lambda x: np.log(x+1))

In [32]:
# applying tanh to squash the values between 0 to 1

In [33]:
def new_sigmoid(x):
    score = (2/(1+np.exp(-x)))-1
    return score

In [34]:
#X_test_standardised['AA'] = X_test['AA'].apply(new_sigmoid)

In [35]:
#X_test_standardised.reset_index(drop=True,inplace=True)

In [36]:
X_test_standardised.head()

,AA,source_parents_count,source_non_leaf_descendants_count,source_leaf_descendans_count,sink_parent_non_leaf_descendants_count,sink_parent_leaf_descendans_count,sink_parents_count
0,0.000000,4.634729,4.189655,2.944439,7.902118,11.604684,1.386294
1,0.341960,3.688879,4.787492,5.402677,12.095543,15.905984,5.669881
2,0.000000,2.639057,4.204693,4.962845,8.130648,9.726512,1.098612
3,0.805984,2.833213,3.891820,3.610918,7.733684,8.302762,3.218876
4,0.589348,3.433987,4.248495,5.293305,10.629900,13.207203,5.111988


In [37]:
X_test_standardised.describe()

,AA,source_parents_count,source_non_leaf_descendants_count,source_leaf_descendans_count,sink_parent_non_leaf_descendants_count,sink_parent_leaf_descendans_count,sink_parents_count
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,0.476025,3.591326,3.716332,4.814037,9.536324,12.900121,3.484307
std,0.729852,1.392287,1.371289,1.938100,2.090113,2.483634,1.777210
min,0.000000,0.693147,0.000000,0.000000,2.197225,2.079442,0.693147
25%,0.000000,2.564949,2.890372,3.610918,8.318071,11.676956,2.079442
50%,0.129048,3.610918,3.784190,4.753590,9.925689,13.632869,3.526361
75%,0.655865,4.574711,4.663439,5.948687,11.101463,14.680368,4.779123
max,6.895624,7.612337,7.683864,12.819595,13.660744,16.560354,8.485083


In [38]:
train_data = pd.read_csv('./PA_AA_train_data.csv')

# Feature matrix for Train data

In [39]:
train_data.drop('PA',axis=1,inplace=True)

In [40]:
train_data.head()

,Source,Sink,AA,value
0,1097,1956386,0.000000,0
1,1097,4240968,0.000000,0
2,1122,1663141,0.077345,0
3,1122,1747030,0.077345,0
4,1122,2962986,25.321024,1


In [41]:
df3 = train_data.join(source_features.set_index('node_id'),on='Source')

In [42]:
df3.head()

,Source,Sink,AA,value,parents_count,non_leaf_descendants_count,leaf_descendans_count
0,1097,1956386,0.000000,0,21,37.0,139.0
1,1097,4240968,0.000000,0,21,37.0,139.0
2,1122,1663141,0.077345,0,109,251.0,761.0
3,1122,1747030,0.077345,0,109,251.0,761.0
4,1122,2962986,25.321024,1,109,251.0,761.0


In [43]:
df3.head()

,Source,Sink,AA,value,parents_count,non_leaf_descendants_count,leaf_descendans_count
0,1097,1956386,0.000000,0,21,37.0,139.0
1,1097,4240968,0.000000,0,21,37.0,139.0
2,1122,1663141,0.077345,0,109,251.0,761.0
3,1122,1747030,0.077345,0,109,251.0,761.0
4,1122,2962986,25.321024,1,109,251.0,761.0


In [44]:
df3.rename(columns= {'parents_count':'source_parents_count',
                              'non_leaf_descendants_count':'source_non_leaf_descendants_count',
                              'leaf_descendans_count':'source_leaf_descendans_count'},inplace=True)

In [45]:
df3.set_index(['Source','Sink'],inplace=True)
df3.head()

AA  value  source_parents_count  \
Source Sink                                              
1097   1956386   0.000000      0                    21   
       4240968   0.000000      0                    21   
1122   1663141   0.077345      0                   109   
       1747030   0.077345      0                   109   
       2962986  25.321024      1                   109   

                source_non_leaf_descendants_count  \
Source Sink                                         
1097   1956386                               37.0   
       4240968                               37.0   
1122   1663141                              251.0   
       1747030                              251.0   
       2962986                              251.0   

                source_leaf_descendans_count  
Source Sink                                   
1097   1956386                         139.0  
       4240968                         139.0  
1122   1663141                         761.0  
       1747030                         761.0  
       2962986                         761.0

In [46]:
df4 = train_data.join(sink_features.set_index('node_id'),on='Sink')

In [47]:
df4.rename(columns={'parents_count':'sink_parents_count',
                              'parent_non_leaf_descendants_count':'sink_parent_non_leaf_descendants_count',
                              'parent_leaf_descendans_count':'sink_parent_leaf_descendans_count'},inplace=True)

In [48]:
df4.set_index(['Source','Sink'],inplace=True)
df4.head()

AA  value  sink_parent_non_leaf_descendants_count  \
Source Sink                                                                
1097   1956386   0.000000      0                                      63   
       4240968   0.000000      0                                    1547   
1122   1663141   0.077345      0                                     472   
       1747030   0.077345      0                                     472   
       2962986  25.321024      1                                   30976   

                sink_parent_leaf_descendans_count  sink_parents_count  
Source Sink                                                            
1097   1956386                                481                   1  
       4240968                             369901                   2  
1122   1663141                             410947                   1  
       1747030                             410947                   1  
       2962986                            1073476                 104

In [49]:
X_train_v2 = pd.concat([df3.sort_index(),df4.sort_index().drop(['value','AA'],axis=1)],axis=1)

In [50]:
X_train_v2.head(10)

AA  value  source_parents_count  \
Source Sink                                              
1097   1956386   0.000000      0                    21   
       4240968   0.000000      0                    21   
1122   1663141   0.077345      0                   109   
       1747030   0.077345      0                   109   
       2962986  25.321024      1                   109   
       3175699   0.000000      0                   109   
       3378788   0.000000      0                   109   
       3396131   4.141560      1                   109   
1152   437467    0.090042      0                    94   
       583373    5.163606      1                    94   

                source_non_leaf_descendants_count  \
Source Sink                                         
1097   1956386                               37.0   
       4240968                               37.0   
1122   1663141                              251.0   
       1747030                              251.0   
       2962986                              251.0   
       3175699                              251.0   
       3378788                              251.0   
       3396131                              251.0   
1152   437467                                99.0   
       583373                                99.0   

                source_leaf_descendans_count  \
Source Sink                                    
1097   1956386                         139.0   
       4240968                         139.0   
1122   1663141                         761.0   
       1747030                         761.0   
       2962986                         761.0   
       3175699                         761.0   
       3378788                         761.0   
       3396131                         761.0   
1152   437467                         2678.0   
       583373                         2678.0   

                sink_parent_non_leaf_descendants_count  \
Source Sink                                              
1097   1956386                                      63   
       4240968                                    1547   
1122   1663141                                     472   
       1747030                                     472   
       2962986                                   30976   
       3175699                                    3550   
       3378788                                    3295   
       3396131                                   15726   
1152   437467                                    14664   
       583373                                    68766   

                sink_parent_leaf_descendans_count  sink_parents_count  
Source Sink                                                            
1097   1956386                                481                   1  
       4240968                             369901                   2  
1122   1663141                             410947                   1  
       1747030                             410947                   1  
       2962986                            1073476                 104  
       3175699                             130914                   6  
       3378788                             396838                   1  
       3396131                             255765                  71  
1152   437467                              911681                  17  
       583373                             3330098                 122

In [51]:
X_train_v2.count()

AA                                        171996
value                                     171996
source_parents_count                      171996
source_non_leaf_descendants_count         171996
source_leaf_descendans_count              171996
sink_parent_non_leaf_descendants_count    171996
sink_parent_leaf_descendans_count         171996
sink_parents_count                        171996
dtype: int64

# Caliculating rank of the train matrix

In [52]:
from numpy.linalg import matrix_rank

In [53]:
X_train_v2.head()

AA  value  source_parents_count  \
Source Sink                                              
1097   1956386   0.000000      0                    21   
       4240968   0.000000      0                    21   
1122   1663141   0.077345      0                   109   
       1747030   0.077345      0                   109   
       2962986  25.321024      1                   109   

                source_non_leaf_descendants_count  \
Source Sink                                         
1097   1956386                               37.0   
       4240968                               37.0   
1122   1663141                              251.0   
       1747030                              251.0   
       2962986                              251.0   

                source_leaf_descendans_count  \
Source Sink                                    
1097   1956386                         139.0   
       4240968                         139.0   
1122   1663141                         761.0   
       1747030                         761.0   
       2962986                         761.0   

                sink_parent_non_leaf_descendants_count  \
Source Sink                                              
1097   1956386                                      63   
       4240968                                    1547   
1122   1663141                                     472   
       1747030                                     472   
       2962986                                   30976   

                sink_parent_leaf_descendans_count  sink_parents_count  
Source Sink                                                            
1097   1956386                                481                   1  
       4240968                             369901                   2  
1122   1663141                             410947                   1  
       1747030                             410947                   1  
       2962986                            1073476                 104

In [54]:
matrix_rank(X_train_v2.values[:, 1:])

7

# Standardising the Train data - Hyperparametre

In [55]:
# The following is a hyperparametre for our model

In [56]:
from sklearn import preprocessing

In [57]:
#X_train_standardised = X_train.drop(['Value','AA'],axis=1).apply(preprocessing.robust_scale)

In [58]:
#X_train_standardised = X_train.drop(['Value'],axis=1).apply(lambda x: np.log10(x+1))

In [59]:
X_train_standardised = X_train_v2.drop(['value'],axis=1).apply(lambda x: np.log(x+1))

In [60]:
#X_train_standardised['AA'] = X_train_v2['AA'].apply(new_sigmoid)

In [61]:
#X_train_standardised.reset_index(drop=True,inplace=True)

In [62]:
X_train_standardised.describe()

,AA,source_parents_count,source_non_leaf_descendants_count,source_leaf_descendans_count,sink_parent_non_leaf_descendants_count,sink_parent_leaf_descendans_count,sink_parents_count
count,171996.000000,171996.000000,171996.000000,171996.000000,171996.000000,171996.000000,171996.000000
mean,0.563048,4.598381,4.730980,6.687036,8.467613,12.567545,2.273194
std,0.923210,1.824378,1.804788,3.058529,2.253250,2.446209,1.732567
min,0.000000,0.693147,0.000000,0.000000,0.000000,1.098612,0.693147
25%,0.000000,3.258097,3.465736,4.369448,6.990947,11.336415,0.693147
50%,0.000000,4.574711,4.753590,6.257668,8.522976,13.137806,1.386294
75%,0.926117,6.063785,6.159095,9.237858,10.328616,14.257465,3.688879
max,9.147485,8.485083,8.522976,13.536809,13.752316,16.573638,8.485083


In [63]:
X_train_standardised.columns

Index(['AA', 'source_parents_count', 'source_non_leaf_descendants_count',
       'source_leaf_descendans_count',
       'sink_parent_non_leaf_descendants_count',
       'sink_parent_leaf_descendans_count', 'sink_parents_count'],
      dtype='object')

In [64]:
X_test_standardised = X_test_standardised[['source_parents_count', 'source_non_leaf_descendants_count',
       'source_leaf_descendans_count',
       'sink_parent_non_leaf_descendants_count',
       'sink_parent_leaf_descendans_count', 'sink_parents_count', 'AA']] 

In [65]:
# The labels 

In [66]:
y_train = X_train_v2['value']

In [67]:
X_train_standardised.shape

(171996, 7)

In [68]:
y_train.sum()

76177

In [69]:
X_test_standardised.head()

,source_parents_count,source_non_leaf_descendants_count,source_leaf_descendans_count,sink_parent_non_leaf_descendants_count,sink_parent_leaf_descendans_count,sink_parents_count,AA
0,4.634729,4.189655,2.944439,7.902118,11.604684,1.386294,0.000000
1,3.688879,4.787492,5.402677,12.095543,15.905984,5.669881,0.341960
2,2.639057,4.204693,4.962845,8.130648,9.726512,1.098612,0.000000
3,2.833213,3.891820,3.610918,7.733684,8.302762,3.218876,0.805984
4,3.433987,4.248495,5.293305,10.629900,13.207203,5.111988,0.589348


# Training the model

In [70]:
from sklearn.linear_model import LogisticRegression

In [71]:
from sklearn.utils import shuffle

In [72]:
X_train, Y_train = shuffle(X_train_standardised, y_train, random_state=0)

In [73]:
X_train['sink_square'] = X_train['sink_parents_count']*X_train['sink_parents_count']

/Users/hima95/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [74]:
X_train.columns

Index(['AA', 'source_parents_count', 'source_non_leaf_descendants_count',
       'source_leaf_descendans_count',
       'sink_parent_non_leaf_descendants_count',
       'sink_parent_leaf_descendans_count', 'sink_parents_count',
       'sink_square'],
      dtype='object')

In [75]:
X_test_standardised['sink_square'] = X_test_standardised['sink_parents_count']*X_test_standardised['sink_parents_count']

In [76]:
X_test_standardised = X_test_standardised[['AA', 'source_parents_count', 'source_non_leaf_descendants_count',
       'source_leaf_descendans_count',
       'sink_parent_non_leaf_descendants_count',
       'sink_parent_leaf_descendans_count', 'sink_parents_count','sink_square']]

In [77]:
logmodel = LogisticRegression(C=1,solver='sag',max_iter=800)

In [78]:
logmodel.fit(X_train,Y_train)

/Users/hima95/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=800, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False)

In [79]:
logmodel.coef_

array([[41.29294448, -0.11122542, -0.28675168,  1.1125546 ,  0.38162356,
        -0.48099864,  3.22459257, -0.40168179]])

In [80]:
from  sklearn.feature_selection import SelectFromModel

In [81]:
SelectFromModel(logmodel.coef_,prefit=True)

SelectFromModel(estimator=array([[41.29294, -0.11123, -0.28675,  1.11255,  0.38162, -0.481  ,
         3.22459, -0.40168]]),
        norm_order=1, prefit=True, threshold=None)

In [82]:
predictions = logmodel.predict(X_test_standardised)

In [83]:
sum(predictions)

1043

In [84]:
len(predictions)

2000

In [85]:
probs = logmodel.predict_proba(X_test_standardised)

In [86]:
probabs = [probs[i][1] for i in range(len(probs))]

In [87]:
logistic_predictions = pd.DataFrame()

In [88]:
logistic_predictions['Prediction'] = probabs

In [89]:
logistic_predictions['Id'] = X_test['Id']

In [90]:
logistic_predictions.set_index('Id',drop=True,inplace=True)

In [91]:
logistic_predictions.to_csv('prediction_trail_31th.csv')

# Train and test split 


In [92]:
from sklearn.model_selection import train_test_split

In [93]:
X_train, X_test, y_train, y_test = train_test_split(X_train, 
                                                    Y_train, test_size=0.30, 
                                                    random_state=117)

In [94]:
logmodel1 = LogisticRegression(C=1,solver= 'sag',max_iter=800)
logmodel1.fit(X_train,y_train)

/Users/hima95/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=800, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False)

In [95]:
logmodel1.coef_

array([[36.62831136, -0.10020888, -0.24697574,  1.08240486,  0.37839876,
        -0.46653499,  3.18102077, -0.3928759 ]])

In [96]:
predictions = logmodel1.predict(X_test)


In [97]:
probs1 = logmodel.predict_proba(X_test)

In [98]:
probabs1 = [probs1[i][1] for i in range(len(probs1))]

In [99]:
from sklearn.metrics import classification_report

In [100]:
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       0.99      0.99      0.99     28755
          1       0.99      0.99      0.99     22844

avg / total       0.99      0.99      0.99     51599



In [101]:
from sklearn import metrics


In [102]:
from sklearn.metrics import roc_auc_score

In [103]:
roc_auc_score(y_test,probabs1)

0.9996886231231368